In [1]:
import os
import datasets
import transformers
from transformers import AutoTokenizer
from huggingface_hub import notebook_login
from dotenv import load_dotenv
load_dotenv(verbose=True)
import torch
import wandb

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
device
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# torch.cuda.is_available()

device(type='cuda', index=0)

In [3]:
seed = 42
wandb.init(
    entity="final_project",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abbymark (use `wandb login --relogin` to force relogin)


# Loading dataset

In [4]:
api_token = os.getenv('USE_AUTH_TOKEN')
dataset = datasets.load_dataset('metamong1/summarization_magazine', use_auth_token=api_token)

Downloading:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/290M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/72.6M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset magazine_summarization downloaded and prepared to /opt/ml/.cache/huggingface/datasets/metamong1___magazine_summarization/Magizine Summarization/1.0.0/506cb41eb0b96b084eafa5dd5fe3b51ff0d1061256700adf1aa92d3b19762c36. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 73640
    })
    validation: Dataset({
        features: ['doc_id', 'title', 'text', 'doc_type', 'file'],
        num_rows: 18411
    })
})

## Checking dataset content

In [6]:
isinstance(dataset, datasets.dataset_dict.DatasetDict) 

True

In [7]:
type(dataset['train'])

datasets.arrow_dataset.Dataset

In [8]:
def show_samples(dataset, num_samples=3, seed=seed):
    if isinstance(dataset, datasets.dataset_dict.DatasetDict):
        sample = dataset['train'].shuffle(seed=seed).select(range(num_samples))
    elif isinstance(dataset, datasets.arrow_dataset.Dataset):
        sample = dataset.shuffle(seed=seed).select(range(num_samples))
    else:
        raise ValueError('Inappropriate dataset.')
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Text: {example['text']}'")

In [9]:
dataset['train'][0]

{'doc_id': 'A201008192682',
 'title': '횡적등방성 매질에서 중합전 역시간 구조보정',
 'text': '역시간 구조보정은 음원영역 파동장 외삽과 수진기영역 파동장 외삽의 상호상관으로 지층구조를 영상화하는 방법으로 복잡한 등방성 매질 층서구조를 영상화하는데 주로 이용된다. 그러나 일반적으로 지구내부 지층구조는 이방성 특성을 지니고 있으므로 이방성을 고려한 구조보정 기술이 필요하다. 여기에서는 편미분 파동장과 음원모음의 내적에 의한 알고리즘과 가상음원과 역전파 파동장과의 내적에 의한 알고리즘을 이용하여 횡적등방성 매질에서 역시간 구조보정 기술을 개발하고자 하였다. 단순 이방성 지층모델에 대한 수치모형실험 결과, 두 가지 방법에 의한 지층단면도 영상은 거의 차이가 없어 가상음원과 역전파 파동장과의 내적으로 구조보정을 실시하는 것이 효과적임을 알 수 있었다. 수평적으로 속도가 변하는 이방성 매질 지층구조에서 편미분 파동장을 구하지 않고 영상화 할 수 있음을 알 수 있었다.',
 'doc_type': '논문',
 'file': '논문요약_0206_0.json'}

In [10]:
show_samples(dataset)

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-93a7be41ce043bd9.arrow



'>> Title: 민주주의 관점으로 본 국가기록관리체계 평가와 전망'
'>> Text: 본 논문은 국가기록관리체계를 민주주의 관점에서 보았다. 지난 10년의 국가기록관리체계를 평가하고 새로운 국가기록관리체계를 전망하기위해서는 민주주의가 우선이 되어야 할 것 같다. 지난 10년의 아카이브가쭈그러진 아카이브였다면, 새로운 아카이브는 어떤 아카이브이어야 할까? 쭈그러진 깡통을 펴듯이 망가지기 전의 상태로 복구하는 게 필요한 일인줄 알면서도 새로운 기록풍경을 그리는 것도 그 못지않게 중요하다고 보았다. 본 논문은 기본적으로 민주주의의 가치를 제도화하는 아카이브 전망에대한 것이다. 나아가 일상적 민주주의에서 아카이브는 무엇을 할 수 있을지도 짚어보았다. 이를 위해 그간의 국가기록관리기구 개편 제안을 체계적으로 검토했다. 그 다음에는 공공기록관 기록관리직의 목소리를 재현해보았다. 기록관리직은 지난 10년 동안 기록공동체와 우리 사회가 일군 사회적자산이기 때문에 그 목소리가 무엇을 원하는지 들어야 한다고 판단했기 때문이다. 기록은 이제 단순히 통치의 수단이 아니라 그 통치의 정통성을 규정하는 통치의 기반이 되었다. 이처럼 기록의 사회적 역할과 의미가 달라졌다. 1999년 시점이 아닌 2017년 시점에서, 국가아카이브의 기록풍경을 다시 그려야 할 것이다. 이런 과제를 위해서는 무엇보다도 민주주의 관점이 필요하다고 생각한다.'

'>> Title: 조선의 대일 교화 양상과 그 기저 (朝鮮의 對日 敎化 樣相과 그 基底)'
'>> Text: 조선전기는 왜구의 잦은 침탈로 인해 일본과의 외교적 해결이 절실히 요구되던 때였다. 그러나 막부의 왜구 통제력 부족으로 교섭이 번번이 무위로 돌아갔다. 그와 같은 상황이 지속되자 명쾌한 대책을 마련하지 못한 조선은 信義를 잃은 일본에 대하여 夷狄으로 대하거나 小國으로 여기는 입장을 취하였다. 이러한 당시의 대일 관계는 문인들에 의해 시문으로 구현되었다. 하지만 시문을 살펴보면, 현안인 왜구를 근절하는 데에만 집중한 나머지 조일 간의 다양한

# Training with sample dataset

### Dividing sample dataset

In [11]:
sample_size = 5000
sample_training_dataset = dataset['train'].shuffle(seed=seed).select(range(sample_size))

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-93a7be41ce043bd9.arrow


In [12]:
show_samples(sample_training_dataset)

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-97e84a9dc97fba8d.arrow



'>> Title: 서방정토 극락세계의 존재에 관한 연구'
'>> Text: 본 논문에서는 서방정토극락세계와 아미타불의 존재와 장엄에 대하여 살펴보았다. 이러한 국토와 부처님은 법장비구의 출가와 발원에서 시작되었다. 그 동기는 이상국가의 건설과 지도자의 탄생을 갈망하는 마음이었다. 48대원의 발원과 오랜 수행으로 극락정토가 이루어졌고, 아미타불이 탄생하였다. 극락정토의 依報莊嚴은 대지와 寶樹와 法音樹, 蓮池, 功德水 등으로 표현되어 있다. 기세간적인 장엄은 물질적인 장엄이다. 극락정토는 물질적은 절대적으로 풍요로운 세계이다. 조금도 부족함이 없는 세계, 완전히 만족한 세계를 표방하고 있다. 그 근거로 각 경전에서 설하는 증거를 제시하여 실존함을 증명코자 하였다. 다음으로 아미타불과 성중들에 대한 正報莊嚴을 살펴보았다. 아미타불은 10겁전에 성불하였으며, 無量光, 無量壽의 부처님이다. 영원한 지혜광명의 부처님이고, 영원한 생명의 부처님인 것이다. 이에 대해 각 경전과 논서들을 증거로 제시하면서 그 존재와 실재를 증명코자 하였다. 이는 정신적인 장엄이며, 안락의 세계이다. 마지막으로 指方立相的인 정토와 아미타불의 존재에 대한 믿음을 밝혔다. 극락정토는 서방으로 십만억불토를 지난 곳에 분명히 존재한다고 정토계 경전에서는 설하고 있다. 그러나 어리석은 중생들은 이에 대해 의혹심을 가지면서 믿고 있다. 이를 반신반의라고 할 수 있을 것이다. 그들은 邊地에 胎生으로 태어난다고 한다. 특히 원효는 정토와 아미타불의 존재를 믿지 않는 것은 부처님의 五智를 의심하는 것이므로 邊地에 태어난다고 하였다. 그러면서 그는 仰信, 應信, 伏信을 설하고 있다. 그 경지는 중생의 알음알이로 알 수 있는 경지가 아니므로 부처님의 말씀을 믿어야 한다고 하였다. 오늘날 불자들 가운데에는 의혹중생이 많으며, 唯心淨土로 이해하여 관념론적인 정토로 해석하는 사람들이 많다. 이는 참으로 경계해야 할 신행방법이라고 아니할 수 없다.'

'>> Title: 북한이탈주민 지원 인력의 소진 요인과 극복방안

### Tokenizer

In [13]:
from transformers import AutoTokenizer

# model_checkpoint = "gogamza/kobart-summarization"
model_checkpoint = 'encoder_decoder_pruned_last_3'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
inputs = tokenizer("제목 생성을 위한 요약 모델을 이제부터 만들어 봅시다!")
inputs

{'input_ids': [12147, 10608, 14106, 14403, 14353, 26200, 24224, 14667, 14150, 14803, 24110, 11465, 9754, 232], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['제',
 '목',
 '▁생',
 '성을',
 '▁위한',
 '▁요약',
 '▁모델을',
 '▁이제',
 '부터',
 '▁만들어',
 '▁봅',
 '시',
 '다',
 '!']

In [16]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['text'], max_length=max_input_length, truncation = True, #padding=True
    )

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['title'], max_length=max_target_length, truncation=True, #padding=True
        )
    
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [17]:
tokenized_datasets = sample_training_dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [18]:
tokenized_eval_datasets = dataset['validation'].select(range(500)).map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

### Baseline score

In [19]:
# from datasets import load_metric
# rouge_score = load_metric('rouge')

import os, sys
sys.path.append('/opt/ml/final-project-level3-nlp-02')
from rouge import compute


In [20]:
generated_summary = ["이번 제목 생성 테스크 진짜 진짜 잘하고 싶다. 두번째줄은 작동 안한다고?", '이번에 잘되면 정말 좋겠다']
reference_summary = ["이번 제복 생성 테스크 잘하고 싶다. 두번째 작동 안한다고?", '이번에 잘되면 좋겠다']
# generated_summary = "I absolutely loved reading the Her Games"
# reference_summary = "I loved reading the Hunger Games"ung
scores = compute(
    predictions=generated_summary, references=reference_summary, tokenizer=tokenizer
)
scores['rouge1'].mid

Score(precision=0.7969924812030075, recall=0.9375, fmeasure=0.8615384615384615)

In [21]:
scores

{'rouge1': AggregateScore(low=Score(precision=0.7368421052631579, recall=0.875, fmeasure=0.7999999999999999), mid=Score(precision=0.7969924812030075, recall=0.9375, fmeasure=0.8615384615384615), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rouge2': AggregateScore(low=Score(precision=0.5555555555555556, recall=0.6666666666666666, fmeasure=0.606060606060606), mid=Score(precision=0.6111111111111112, recall=0.7333333333333334, fmeasure=0.6666666666666665), high=Score(precision=0.6666666666666666, recall=0.8, fmeasure=0.7272727272727272)),
 'rougeL': AggregateScore(low=Score(precision=0.7368421052631579, recall=0.875, fmeasure=0.7999999999999999), mid=Score(precision=0.7969924812030075, recall=0.9375, fmeasure=0.8615384615384615), high=Score(precision=0.8571428571428571, recall=1.0, fmeasure=0.923076923076923)),
 'rougeLsum': AggregateScore(low=Score(precision=0.7368421052631579, recall=0.875, fmeasure=0.7999999999999999), mid=Score(precision=0.796992

In [22]:
import kss
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

def one_sentence_title(text):
    return kss.split_sentences(text)[0]

In [23]:
print(one_sentence_title(sample_training_dataset['text'][0]))

[Korean Sentence Splitter]: Initializing Pynori...


본 논문은 국가기록관리체계를 민주주의 관점에서 보았다.


In [24]:
def evaluate_baseline(dataset, tokenizer):
    summaries = [one_sentence_title(text) for text in dataset['text']]
    return compute(predictions=summaries, references=dataset['title'], tokenizer=tokenizer)

In [25]:
import pandas as pd

score = evaluate_baseline(dataset['validation'].shuffle().select(range(10)), tokenizer)
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 26.39, 'rouge2': 14.87, 'rougeL': 23.42, 'rougeLsum': 23.5}

## Fine Tuning

In [26]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [27]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 2

# Show the training loss with every epoch
logging_steps = 100 #len(tokenized_datasets) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    # output_dir=f'{model_name}-finetuned-paper-sample-size-1000',
    output_dir=f'{model_name}-finetuned-paper-sample-size-1000-pruned',
    evaluation_strategy='steps', #'epoch',
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    save_total_limit=2,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    report_to='wandb'
    # push_to_hub=True,
)

In [28]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    # decoded_preds = ['\n'.join(kss.split_sentences(pred.strip())) for pred in decoded_preds]
    # decoded_labels = ['\n'.join(kss.split_sentences(label.strip())) for label in decoded_labels]

    # Compute ROUGE scores
    result = compute(
        predictions=decoded_preds, references=decoded_labels, tokenizer=tokenizer
    )

    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [29]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [30]:
len(tokenized_datasets[5]['attention_mask'])

325

In [31]:
columns_to_return = ['input_ids', 'labels', 'attention_mask']
tokenized_datasets.set_format(type='torch', columns = columns_to_return)#, device=device)


In [79]:
features = [tokenized_datasets[i] for i in range(2)]
data_collator(features)

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'input_ids': tensor([[10888, 14396, 19255,  ...,     3,     3,     3],
        [16446, 12126, 14575,  ...,  3020, 18037, 14098]]), 'labels': tensor([[15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044, 14653,
         11863, 14718,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [16446, 12024, 14029, 12041, 14139, 13714, 14256, 17664, 14028, 14040,
         12123, 14338,  4543,  8813, 12024,  1700,  3486,  4400,  1700,  4336,
          2682,  1700,  4823,  5996,  9120, 14028,  1700,  3095,  3704,   240]]), 'decoder_input_ids': tensor([[    2, 15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044,
         14653, 11863, 14718,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 16446, 12024, 140

In [115]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_eval_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

In [118]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_type, title, text, doc_id, file.
***** Running training *****
  Num examples = 5000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,2.018900,3.523563,29.764400,16.678900,26.977500,27.042100
200,2.022400,3.523563,29.740300,16.622000,27.049700,27.022300
300,2.007000,3.523563,29.795800,16.621600,27.007700,26.988000
400,2.017400,3.523563,29.807100,16.603000,27.015200,26.958600
500,2.043000,3.523563,29.732300,16.609000,27.006800,26.964100
600,2.025700,3.523563,29.698400,16.600600,27.014700,26.995100
700,1.944200,3.523563,29.731900,16.636900,27.000700,26.979000
800,2.035500,3.523563,29.720000,16.636300,27.013600,26.984800
900,1.974700,3.523563,29.801300,16.634700,27.031200,27.017000
1000,2.011800,3.523563,29.787500,16.644500,27.056000,26.980900


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_type, title, text, doc_id, file.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_type, title, text, doc_id, file.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_type, title, text, doc_id, file.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_type, title, text, doc_id, file.
***** Running Evaluation *****
  Num examples = 500
  Batch s

TrainOutput(global_step=1250, training_loss=2.0162662963867186, metrics={'train_runtime': 244.8054, 'train_samples_per_second': 40.849, 'train_steps_per_second': 5.106, 'total_flos': 1418176574521344.0, 'train_loss': 2.0162662963867186, 'epoch': 2.0})

In [27]:
trainer.evaluate()

NameError: name 'trainer' is not defined

# Uing Accelerate(Raw pytorch)

In [58]:
tokenized_datasets.set_format("torch")

In [59]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/gogamza/kobart-summarization/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/1c32baaf6a1067a5e27a0dfbac0a3d23a86d958ab10b092d5ea4150bd451de17.4e52ef6c87e6938c92ba0d19888607d76e30e950e81060a8fa6cb1189c93614d
/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_

In [60]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

eval_dataloader = DataLoader(
    tokenized_eval_datasets,
    collate_fn=data_collator,
    batch_size=batch_size
)

In [61]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [64]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 2
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [65]:
def postprocess_text(preds, labels):
    preds = [preds.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch['input_ids'],
                attention_mask=batch['attention_mask'],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch['labels']

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch['labels'], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_toekns = True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_toekns=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

    #         rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)
    
    # # Compute metrics
    # result = rouge_score.compute()

    # # Extract the median ROUGE scores
    # result = {key: value.mid.f}#####################################################################
    

# Using Fine-tuned model

In [2]:
tokenizer= AutoTokenizer.from_pretrained('kobart-summarization-finetuned-paper-sample-size-1000/checkpoint-1000')

NameError: name 'AutoTokenizer' is not defined

In [2]:
from transformers import pipeline

summzrizer = pipeline('summarization', model='kobart-summarization-finetuned-paper-sample-size-1000/checkpoint-1000')

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [43]:
idx = 7000
title = dataset['validation'][idx:idx+10]['title']
text = dataset['validation'][idx:idx+10]['text']

In [44]:
title

['우리나라 기능직공무원의 직무만족에 미치는 요인 및 영향 분석',
 '아파트시장의 네트워크 효과',
 '한반도의 지정학적 의미에 대한 중국 정치엘리트의 전통적 인식',
 '한류열풍 속 대중문화를 통한 아시아 선교의 가능성',
 '혐기소화공정에서 항생항균물질이 메탄생성에 미치는 영향',
 '일본 공립도서관의 유형별 특성 분석 - 서비스내용과 서비스공간의 연계적 고찰을 중심으로 -',
 '소셜데이터 분석 및 인공지능 알고리즘 기반 범죄 수사 기법 연구',
 '소프트웨어 보안약점의 중요도에 대한 정량 평가 기준 연구',
 '동북아 안보환경에서 스텔스 전투기의 전략적 유용성과 한국 공군에 주는 함의',
 '기계설계분야 중견 엔지니어의 일과 학습에 관한 내러티브 연구: 엔지니어의 직무관련학습의 맥락과 공학교육에 대한 시사점 찾기']

In [ ]:
text

In [3]:
summzrizer('근대 경제학에서는 행위의 합리성 문제를 비용 대 편익이라는 효율성 측면에서 접근하는데, 이러한 사고방식은 경제학을 넘어 사회과학 전반으로 널리 확산되 었다. 하지만 경제적 논리로는 설명하기 어렵거나 바람직하지 않은 결과를 낳은 현상도 많이 존재한다. 특히 도덕이나 규범은 경제 현상의 작동 방식에도 영향을 미치기 때문에 이를 설명하는 확장된 경제 이론이 필요하다. 도덕경제론은 바로 이러한 문제를 다루는 접근 방식이다. 이에 따르면 경제 활동에서 개인의 효용 극대화와 합리적 선택을 강조하는 경제학의 기본 가정 역시 초시대적으로 통용 되는 가치가 아니라 자본주의 등장 이후에 형성된 역사적 현상에 불과하다. 경제 적 원칙은 당대의 도덕적 규범에 의해 구성되는 상대적인 가치이기 때문에, 경제 활동이나 영역과 관련된 규범적 측면을 고려해 재구성해야 한다. ‘도덕경제 (moral economy)’론은 바로 이처럼 경제 현상에서 규범이나 문화의 역할 문제 를 다루는 접근 방식이다. 이러한 시도는 경제와 관련된 사회 현상을 이해하는 데 에서 경제학적 접근 방식의 편협성과 한계를 해결하는 데에도 풍부한 시사점을 줄수있을것이다. 이논문은기존의도덕경제론에서다룬주요쟁점과개념을 소개하고, 이 논의가 미디어 경제를 이해하는 데 주는 함의, 쟁점 등을 검토한다.')

[{'summary_text': "미디어 경제에서의 '도덕경제론' 이해 - - - - - - - - -"}]

In [9]:
compute(
    predictions=['도덕경제론과 미디어 경제 연구'], references=["미디어 경제에서의 '도덕경제론' 이해 - - - - - - - - -"], tokenizer=tokenizer
)

{'rouge1': AggregateScore(low=Score(precision=0.42857142857142855, recall=0.16666666666666666, fmeasure=0.24), mid=Score(precision=0.42857142857142855, recall=0.16666666666666666, fmeasure=0.24), high=Score(precision=0.42857142857142855, recall=0.16666666666666666, fmeasure=0.24)),
 'rouge2': AggregateScore(low=Score(precision=0.16666666666666666, recall=0.058823529411764705, fmeasure=0.08695652173913045), mid=Score(precision=0.16666666666666666, recall=0.058823529411764705, fmeasure=0.08695652173913045), high=Score(precision=0.16666666666666666, recall=0.058823529411764705, fmeasure=0.08695652173913045)),
 'rougeL': AggregateScore(low=Score(precision=0.2857142857142857, recall=0.1111111111111111, fmeasure=0.16), mid=Score(precision=0.2857142857142857, recall=0.1111111111111111, fmeasure=0.16), high=Score(precision=0.2857142857142857, recall=0.1111111111111111, fmeasure=0.16)),
 'rougeLsum': AggregateScore(low=Score(precision=0.2857142857142857, recall=0.1111111111111111, fmeasure=0.16

# Testing

In [1]:
text = '근대 경제학에서는 행위의 합리성 문제를 비용 대 편익이라는 효율성 측면에서 접근하는데, 이러한 사고방식은 경제학을 넘어 사회과학 전반으로 널리 확산되 었다. 하지만 경제적 논리로는 설명하기 어렵거나 바람직하지 않은 결과를 낳은 현상도 많이 존재한다. 특히 도덕이나 규범은 경제 현상의 작동 방식에도 영향을 미치기 때문에 이를 설명하는 확장된 경제 이론이 필요하다. 도덕경제론은 바로 이러한 문제를 다루는 접근 방식이다. 이에 따르면 경제 활동에서 개인의 효용 극대화와 합리적 선택을 강조하는 경제학의 기본 가정 역시 초시대적으로 통용 되는 가치가 아니라 자본주의 등장 이후에 형성된 역사적 현상에 불과하다. 경제 적 원칙은 당대의 도덕적 규범에 의해 구성되는 상대적인 가치이기 때문에, 경제 활동이나 영역과 관련된 규범적 측면을 고려해 재구성해야 한다. ‘도덕경제 (moral economy)’론은 바로 이처럼 경제 현상에서 규범이나 문화의 역할 문제 를 다루는 접근 방식이다. 이러한 시도는 경제와 관련된 사회 현상을 이해하는 데 에서 경제학적 접근 방식의 편협성과 한계를 해결하는 데에도 풍부한 시사점을 줄수있을것이다. 이논문은기존의도덕경제론에서다룬주요쟁점과개념을 소개하고, 이 논의가 미디어 경제를 이해하는 데 주는 함의, 쟁점 등을 검토한다.'


In [99]:
input = tokenizer(text, return_tensors='pt')

In [104]:
label = tokenizer('천하통일', return_tensors='pt')

In [106]:
label

{'input_ids': tensor([[12673, 13586, 21780]]), 'attention_mask': tensor([[1, 1, 1]])}

In [163]:
output = model(**input.to(device), labels=label['input_ids'].to(device))#, output_hidden_states=True, output_attentions=True)

In [164]:
len(output)

4

In [ ]:
output.decoder_attentions

In [160]:
output.decoder_hidden_states[0].shape

torch.Size([1, 3, 768])

In [ ]:
input.to(device)

In [ ]:
tokenized_datasets.select(range(1))[0]

In [83]:
data_collator(features)

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'input_ids': tensor([[10888, 14396, 19255,  ...,     3,     3,     3],
        [16446, 12126, 14575,  ...,  3020, 18037, 14098]]), 'labels': tensor([[15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044, 14653,
         11863, 14718,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [16446, 12024, 14029, 12041, 14139, 13714, 14256, 17664, 14028, 14040,
         12123, 14338,  4543,  8813, 12024,  1700,  3486,  4400,  1700,  4336,
          2682,  1700,  4823,  5996,  9120, 14028,  1700,  3095,  3704,   240]]), 'decoder_input_ids': tensor([[    2, 15352, 14453, 14073, 16439, 14285, 14518, 17469, 15095, 20044,
         14653, 11863, 14718,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 16446, 12024, 140

In [85]:
model.to(device)
output = model(**data_collator(features).to(device))

In [86]:
print(output.loss)

tensor(9.8253, device='cuda:0', grad_fn=<NllLossBackward0>)


In [96]:
output[3][0][0][0]

tensor(0.5473, device='cuda:0', grad_fn=<SelectBackward0>)